In [18]:
import pandas as pd

# 데이터 불러오기
data = pd.read_csv('WallThining_data.csv')

# 데이터 확인
print(data.head())

  Thining Part Moment Direction Thining Rate Boundary Type  Inner Pressure  \
0        Extra              IPO          20%    Limit Load          22.070   
1        Extra              IPO          20%    Limit Load          19.863   
2        Extra              IPO          20%    Limit Load          17.656   
3        Extra              IPO          20%    Limit Load          15.449   
4        Extra              IPO          20%    Limit Load          13.242   

       Moment  Multiplier  
0         0.0    0.754368  
1  17500000.0    0.889164  
2  35000000.0    1.000607  
3  52500000.0    1.022539  
4  70000000.0    0.970476  


In [23]:
len(data)

594

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler

# Categorical 변수 인코딩
le = LabelEncoder()
data['Thining Part'] = le.fit_transform(data['Thining Part'])
data['Moment Direction'] = le.fit_transform(data['Moment Direction'])
data['Boundary Type'] = le.fit_transform(data['Boundary Type'])

# Numerical 변수 전처리 (Inner pressure)
data['Thining Rate'] = data['Thining Rate'].str.rstrip('%').astype('int')

# 데이터 분할
X = data.drop('Multiplier', axis=1).drop('Moment', axis=1)
y = data['Multiplier']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [22]:
import numpy as np
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

models = {
    'Linear Regression': LinearRegression(),
    'Decision Tree Regressor': DecisionTreeRegressor(),
    'Random Forest Regressor': RandomForestRegressor(),
    'Gradient Boosting Regressor': GradientBoostingRegressor(),
    'Ridge Regression': Ridge(),
    'Lasso Regression': Lasso(),
    'SVR': SVR()
}

# MAPE와 SMAPE 함수 정의
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def symmetric_mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs(y_true - y_pred) / ((np.abs(y_true) + np.abs(y_pred)) / 2)) * 100

# 모델 학습 및 성능 평가
for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    mape = mean_absolute_percentage_error(y_test, y_pred)
    smape = symmetric_mean_absolute_percentage_error(y_test, y_pred)
    
    print(f'{name} - MAE: {mae:.4f}, MSE: {mse:.4f}, RMSE: {rmse:.4f}, MAPE: {mape:.4f}, SMAPE: {smape:.4f}')


Linear Regression - MAE: 0.0786, MSE: 0.0090, RMSE: 0.0946, MAPE: 12.2213, SMAPE: 11.9280
Decision Tree Regressor - MAE: 0.0181, MSE: 0.0011, RMSE: 0.0326, MAPE: 2.7316, SMAPE: 2.7551
Random Forest Regressor - MAE: 0.0170, MSE: 0.0007, RMSE: 0.0266, MAPE: 2.5660, SMAPE: 2.5055
Gradient Boosting Regressor - MAE: 0.0170, MSE: 0.0006, RMSE: 0.0243, MAPE: 2.6051, SMAPE: 2.5767
Ridge Regression - MAE: 0.0786, MSE: 0.0089, RMSE: 0.0946, MAPE: 12.2205, SMAPE: 11.9258
Lasso Regression - MAE: 0.1168, MSE: 0.0199, RMSE: 0.1412, MAPE: 19.3384, SMAPE: 18.2369
SVR - MAE: 0.0637, MSE: 0.0057, RMSE: 0.0756, MAPE: 10.5638, SMAPE: 10.1311
